# Custom Layers

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import backend as K

2023-12-07 19:46:48.052757: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Lambda Layers

### Prepare Dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Build Model

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Lambda(lambda x: tf.abs(x)), 
  tf.keras.layers.Dense(10, activation='softmax')
])

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2242 - accuracy: 0.9369
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0909 - accuracy: 0.9732
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0633 - accuracy: 0.9806
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0476 - accuracy: 0.9851
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.0851 - accuracy: 0.9764


[0.08513845503330231, 0.9764000177383423]

In [5]:
def my_leaky_relu(x):
    return K.maximum(-0.01*x, x)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(my_leaky_relu), 
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2563 - accuracy: 0.9270
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1120 - accuracy: 0.9670
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0771 - accuracy: 0.9767
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0581 - accuracy: 0.9821
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.0704 - accuracy: 0.9775


[0.07037085294723511, 0.9775000214576721]

## Implementing Custom Layers

In [6]:
from tensorflow.keras.layers import Layer

In [7]:
class SimpleDense(tf.keras.layers.Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype=tf.float32),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype=tf.float32),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

In [8]:
# declare an instance of the class
my_dense = SimpleDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

[<tf.Variable 'simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.04272778]], dtype=float32)>, <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [9]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

xs = xs.reshape(xs.shape[0], 1)

# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=0)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)

1/1 [==============================] - 0s 69ms/step
[[18.981474]]
[<tf.Variable 'simple_dense_1/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.997315]], dtype=float32)>, <tf.Variable 'simple_dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9916759], dtype=float32)>]


## Implementing Custom Layers with Activation Function

In [10]:
class SimpleDense(tf.keras.layers.Layer):

    def __init__(self, units=32, activation=None):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype=tf.float32),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype=tf.float32),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [11]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  SimpleDense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2607 - accuracy: 0.9261
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1158 - accuracy: 0.9657
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0804 - accuracy: 0.9756
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0608 - accuracy: 0.9818
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.0771 - accuracy: 0.9763


[0.07705428451299667, 0.9763000011444092]